In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# --------------------------- INITIALIZE CSV FILES IF NOT EXISTS ---------------------------
def initialize_csv_files():
    if not os.path.exists("Users.csv"):
        pd.DataFrame(columns=["User ID", "User Name", "Password"]).to_csv("Users.csv", index=False)
    if not os.path.exists("Owners.csv"):
        pd.DataFrame(columns=["Owner ID", "Owner Name", "Password"]).to_csv("Owners.csv", index=False)
    if not os.path.exists("Cars.csv"):
        pd.DataFrame(columns=["Car No.", "Car Name", "Brand", "Branch", "Fuel Type", "Cost", "Category"]).to_csv("Cars.csv", index=False)
    if not os.path.exists("Members.csv"):
        pd.DataFrame(columns=["MID", "M Name", "Phone No.", "No. of cars Booked"]).to_csv("Members.csv", index=False)
    if not os.path.exists("Cars Booked.csv"):
        pd.DataFrame(columns=["Car Name", "M Name", "Date of Booking", "Number of Days", "Total Cost"]).to_csv("Cars Booked.csv", index=False)

initialize_csv_files()

# --------------------------- USER CLASS ---------------------------
class User:
    def showCars(self):
        df = pd.read_csv("Cars.csv")
        print(df)

    def searchCar(self):
        carname = input("Enter a Car name:")
        df = pd.read_csv("Cars.csv")
        result = df.loc[df["Car Name"] == carname]
        if result.empty:
            print("No cars found with the given name")
        else:
            print(result)

    def bookCar(self):
        carname = input("Enter car name:")
        df_cars = pd.read_csv("Cars.csv")
        car = df_cars.loc[df_cars["Car Name"] == carname]
        if car.empty:
            print("Car not found")
            return
        mname = input("Enter member name:")
        df_members = pd.read_csv("Members.csv")
        member = df_members.loc[df_members["M Name"] == mname]
        if member.empty:
            print("Member not found")
            return
        date_of_booking = input("Enter date of booking (YYYY-MM-DD):")
        num_days = int(input("Enter number of days booked:"))
        cost_per_day = car["Cost"].values[0]
        total_cost = num_days * cost_per_day
        print("-" * 40)
        print(f"Booking Summary:\nCar: {carname}\nMember: {mname}\nCost/Day: {cost_per_day}\nTotal Cost: {total_cost}")
        print("-" * 40)
        confirm = input("Confirm booking? (yes/no): ")
        if confirm.lower() == "yes":
            df_booked = pd.read_csv("Cars Booked.csv")
            df_booked.loc[len(df_booked)] = [carname, mname, date_of_booking, num_days, total_cost]
            df_booked.to_csv("Cars Booked.csv", index=False)
            print("Car booked successfully!")
        else:
            print("Booking cancelled")

    def returnCar(self):
        mname = input("Enter member name:")
        carname = input("Enter car name:")
        df_booked = pd.read_csv("Cars Booked.csv")
        booking = df_booked[(df_booked["Car Name"] == carname) & (df_booked["M Name"] == mname)]
        if booking.empty:
            print("No booking found for this car and member")
        else:
            confirm = input("Confirm return? (yes/no): ")
            if confirm.lower() == "yes":
                df_booked = df_booked.drop(booking.index)
                df_booked.to_csv("Cars Booked.csv", index=False)
                print("Car returned successfully!")
            else:
                print("Return cancelled")

# --------------------------- BUSINESS OWNER CLASS ---------------------------
class BusinessOwner:
    def addNewMember(self):
        mid = int(input("Enter member ID:"))
        mname = input("Enter member name:")
        phoneno = input("Enter phone number:")
        df_members = pd.read_csv("Members.csv")
        df_members.loc[len(df_members)] = [mid, mname, phoneno, 0]
        df_members.to_csv("Members.csv", index=False)
        print("Member added successfully!")

    def deleteMember(self):
        mid = int(input("Enter member ID to delete:"))
        df_members = pd.read_csv("Members.csv")
        df_members = df_members.drop(df_members[df_members["MID"] == mid].index)
        df_members.to_csv("Members.csv", index=False)
        print("Member deleted successfully")

    def showbookedCars(self):
        df_booked = pd.read_csv("Cars Booked.csv")
        print(df_booked)
        print("-" * 50)
        for _, row in df_booked.iterrows():
            print(f"Car: {row['Car Name']}\tMember: {row['M Name']}\tTotal Cost: {row['Total Cost']}")
        print("-" * 50)

# --------------------------- LOGIN FUNCTIONS ---------------------------
def loginUser():
    uid = input("Enter User ID: ")
    uname = input("Enter User Name: ")
    pwd = input("Enter Password: ")
    df = pd.read_csv("Users.csv")
    user_row = df[(df["User ID"] == uid) & (df["User Name"] == uname) & (df["Password"] == pwd)]
    if user_row.empty:
        print("Invalid user credentials")
        return None
    print("User login successful")
    return User()

def loginBusinessOwner():
    oid = input("Enter Owner ID: ")
    oname = input("Enter Owner Name: ")
    pwd = input("Enter Password: ")
    df = pd.read_csv("Owners.csv")
    owner_row = df[(df["Owner ID"] == oid) & (df["Owner Name"] == oname) & (df["Password"] == pwd)]
    if owner_row.empty:
        print("Invalid owner credentials")
        return None
    print("Business Owner login successful")
    return BusinessOwner()

# --------------------------- MAIN PROGRAM ---------------------------
print("\nWELCOME TO LUXURY CAR RENTALS")
print("1. Login as User")
print("2. Login as Business Owner")
role_choice = int(input("Enter your choice: "))

if role_choice == 1:
    user_obj = loginUser()
    if user_obj:
        while True:
            print("\n--- USER MENU ---")
            print("1. Show Cars")
            print("2. Search Car")
            print("3. Book Car")
            print("4. Return Car")
            print("5. Exit")
            choice = int(input("Enter choice: "))
            if choice == 1:
                user_obj.showCars()
            elif choice == 2:
                user_obj.searchCar()
            elif choice == 3:
                user_obj.bookCar()
            elif choice == 4:
                user_obj.returnCar()
            elif choice == 5:
                break
            else:
                print("Invalid option")
elif role_choice == 2:
    owner_obj = loginBusinessOwner()
    if owner_obj:
        while True:
            print("\n--- BUSINESS OWNER MENU ---")
            print("1. Add Member")
            print("2. Delete Member")
            print("3. Show Booked Cars")
            print("4. Exit")
            choice = int(input("Enter choice: "))
            if choice == 1:
                owner_obj.addNewMember()
            elif choice == 2:
                owner_obj.deleteMember()
            elif choice == 3:
                owner_obj.showbookedCars()
            elif choice == 4:
                break
            else:
                print("Invalid option")
else:
    print("Invalid role selection")

print("THANK YOU FOR VISITING LUXURY CAR RENTALS")



WELCOME TO LUXURY CAR RENTALS
1. Login as User
2. Login as Business Owner


Enter your choice:  2
Enter Owner ID:  11
Enter Owner Name:  Rohan Chaudhari
Enter Password:  Rohan2003


Invalid owner credentials
THANK YOU FOR VISITING LUXURY CAR RENTALS
